In [75]:
import requests
import regex as re
import pandas as pd
import pickle
from bs4 import BeautifulSoup

import nltk
nltk.download('stopwords')
from sklearn.naive_bayes import MultinomialNB
import xlrd
import string
# from nltk.corpus import stopwords
# nltk.download('stopwords') 
from random import shuffle
import csv

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/stephaniedavis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [67]:
def remove_punctuation(text):
    return re.sub(u"\p{P}+", "", text)


def data_clean(line):
    l = []
    line = line.split()
    for i in line:
        i = i.lower()
        if '>' in i:
            i = i.strip('>')
        if i[0] != '@' and i[0] != "#":
            i = remove_punctuation(i)
            if len(i) != 0  and i[0] != '@' and i[:5] != 'https' and i not in stop_words and i != 'rt' and i != 'amp':
                l.append(i)
    return l

In [68]:
book_df = pd.read_csv('books.csv')
book_df.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [73]:
stop_words = set(stopwords.words('english'))
book_dict = {}
for i in range(0, 999):
    url = 'https://www.goodreads.com/book/show/'+str(book_df.iloc[i]['book_id'])+'.json?key=V0XBUTEOxqia8uVBut2w'
    r = requests.get(url)
    doc = r.text
    soup = BeautifulSoup(doc)
    doc_string = ""

    for d in soup.find_all('description'):
        doc_string += d.text

    for t in soup.find_all('shelf'):
        doc_string = doc_string + " " + " ".join(str(t.get("name")).split("-"))

    book_dict[book_df.iloc[i]['book_id']] = data_clean(doc_string)

    
# results stored in book_dict
# print(book_dict)

In [76]:
with open('thousand_dict.pickle', 'wb') as handle:
    pickle.dump(book_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)